In [1]:
import os

from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import pandas as pd
import numpy as np
import json

/opt/conda/envs/openmmlab/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


# Get Prediction from MMSeg

## Need to change MMseg code 


### mmseg/model/segmentors/encoder_decoder.py, code line 282 
### seg_pred = seg_logit.argmax(dim=1) --> seg_pred = seg_logit 


In [2]:
# config file 들고오기
cfg = Config.fromfile('/opt/ml/input/mmsegmentation/configs/_swinL_/_base_/base_split3.py')
root='/opt/ml/input/data/mmseg/images/test'
epoch = 'best_mIoU_epoch_64'

# dataset config 수정
cfg.data.test.img_dir = root
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1

cfg.work_dir = '/opt/ml/input/mmsegmentation/work_dirs/base_split3'

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.samples_per_gpu,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

model1 = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model1, checkpoint_path, map_location='cpu')

model1.CLASSES = dataset.CLASSES
model1 = MMDataParallel(model1.cuda(), device_ids=[0])

2023-01-05 02:03:45,893 - mmseg - INFO - Loaded 819 images
/opt/ml/input/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:276: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: /opt/ml/input/mmsegmentation/work_dirs/base_split3/best_mIoU_epoch_64.pth


In [3]:
# config file 들고오기
cfg = Config.fromfile('/opt/ml/input/mmsegmentation/configs/_swinL_/_base_/base_split2_pseudo.py')
root='/opt/ml/input/data/mmseg/images/test'
epoch = 'best_mIoU_epoch_67'

# dataset config 수정
cfg.data.test.img_dir = root
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1

cfg.work_dir = '/opt/ml/input/mmsegmentation/work_dirs/base_split2_pseudo'

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.samples_per_gpu,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

model2 = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model2, checkpoint_path, map_location='cpu')

model2.CLASSES = dataset.CLASSES
model2 = MMDataParallel(model2.cuda(), device_ids=[0])

2023-01-05 02:03:53,342 - mmseg - INFO - Loaded 819 images


load checkpoint from local path: /opt/ml/input/mmsegmentation/work_dirs/base_split2_pseudo/best_mIoU_epoch_67.pth


In [4]:
# config file 들고오기
cfg = Config.fromfile('/opt/ml/input/mmsegmentation/configs/_swinL_/_base_/base_split1_pseudo.py')
root='/opt/ml/input/data/mmseg/images/test'
epoch = 'epoch_70'

# dataset config 수정
cfg.data.test.img_dir = root
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1

cfg.work_dir = '/opt/ml/input/mmsegmentation/work_dirs/base_split1_pseudo'

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.samples_per_gpu,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

model3 = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model3, checkpoint_path, map_location='cpu')

model3.CLASSES = dataset.CLASSES
model3 = MMDataParallel(model3.cuda(), device_ids=[0])

2023-01-05 02:05:21,374 - mmseg - INFO - Loaded 819 images


load checkpoint from local path: /opt/ml/input/mmsegmentation/work_dirs/base_split1_pseudo/best_mIoU_epoch_44.pth


In [5]:
output_mmseg1 = single_gpu_test(model1, data_loader)
print("done")

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 0.5 task/s, elapsed: 1502s, ETA:     0sdone


In [6]:
output_mmseg2 = single_gpu_test(model2, data_loader)
print("done")

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 0.5 task/s, elapsed: 1506s, ETA:     0sdone


In [7]:
output_mmseg3 = single_gpu_test(model3, data_loader)
print("done")

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 0.5 task/s, elapsed: 1516s, ETA:     0sdone


# Get Prediction from baseline

In [ ]:
# import sys

# sys.path.append("/opt/ml/segmentation/baseline_code")

In [ ]:
# import os
# import torch
# import pandas as pd
# import numpy as np
# import yaml
# import ttach as tta

# from tools import test
# from data_set.data_augmentation import get_transform
# from data_set.data_set import CustomDataSet, collate_fn
# from model.custom_encoder import register_encoder
# import segmentation_models_pytorch as smp

# import torch.nn.functional as F

# @torch.no_grad()
# def inference(cfg, device):

#     register_encoder()

#     # TTA transform
#     tta_transforms = tta.Compose(
#     [
#         tta.HorizontalFlip(),
#         tta.VerticalFlip(),
#         tta.Scale(scales=[0.5, 0.75, 1.0, 1.25, 1.5])
#     ])
    
#     dataset_path  = '../input/data'
#     test_path = '../input/data/test.json'
#     test_transform = get_transform('test_transform')
#     test_dataset = CustomDataSet(data_dir=test_path, dataset_path=dataset_path, mode='test', transform=test_transform)
#     test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
#                                           batch_size=16,
#                                           shuffle=False,
#                                           num_workers=4,
#                                           collate_fn=collate_fn)


#     model_path =  f"{cfg['saved_dir']}/{cfg['exp_name']}/{cfg['exp_name']}.pt"                                    
    
#     # model 불러오기
#     model = smp.__dict__[cfg['model']['name']]
#     model = model(encoder_weights=None, **cfg['model']['params'])
#     # best model 불러오기
#     checkpoint = torch.load(model_path, map_location=device)
#     model.load_state_dict(checkpoint)


#     model = model.to(device)
#     # 추론을 실행하기 전에는 반드시 설정 (batch normalization, dropout 를 평가 모드로 설정)
#     model.eval()

#     tta_model = tta.SegmentationTTAWrapper(model, tta_transforms, merge_mode='mean')

#     # test set에 대한 prediction
#     preds = test(tta_model, test_loader, device)
#     preds = F.softmax(preds, dim=1)
#     preds = preds.cpu().numpy()
#     preds = list(preds)

#     return preds

In [ ]:
# yaml_path = "/opt/ml/segmentation/baseline_code/saved/PAN_Swin_Alldata/PAN_Swin_Alldata.yaml"

# with open(yaml_path) as f:
#     cfg = yaml.safe_load(f)

# device = "cuda" if torch.cuda.is_available() else "cpu"

# output_base = inference(cfg, device)

# Ensemble, soft voting

In [8]:
output = []

for predict1, predict2, predict3 in zip(output_mmseg1, output_mmseg2, output_mmseg3):
    predict = predict1 + predict2 + predict3
    predict = predict.argmax(axis=0)
    output.append(predict)

output = np.array(output)
print(output.shape)

(819, 512, 512)


## Save mean login to pickle file

In [10]:
mean_logit = []

for predict1, predict2, predict3 in zip(output_mmseg1, output_mmseg2, output_mmseg3):
    predict = predict1 + predict2 + predict3
    predict = predict/3
    mean_logit.append(predict)

# mean_logit = np.array(mean_logit)
# print(mean_logit.shape)

In [11]:
import pickle
with open('swinL_uper_split123_pseudo_update.pickle','wb') as f:
    pickle.dump(mean_logit, f, pickle.HIGHEST_PROTOCOL)
# with open('fold4.pickle','rb') as f:
#     output_mmseg = pickle.load(f)

In [ ]:
output_mmseg3

# Make CSV

In [13]:
# sample_submisson.csv 열기
submission = pd.read_csv('/opt/ml/input/code/submission/sample_submission.csv', index_col=None)
json_dir = os.path.join("/opt/ml/input/data/test.json")
with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

input_size = 512
output_size = 256
bin_size = input_size // output_size

# PredictionString 대입
for image_id, predict in enumerate(output):
    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    
    temp_mask = []
    predict = predict.reshape(1, 512, 512)
    mask = predict.reshape((1, output_size, bin_size, output_size, bin_size)).max(4).max(2) # resize to 256*256
    temp_mask.append(mask)
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)

    string = oms.flatten()

    # submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
    #                                ignore_index=True)
    submission = pd.concat([submission, pd.DataFrame([{"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}])]
                                   , ignore_index=True)

# submission.csv로 저장
submission.to_csv(os.path.join('/opt/ml', f'ensemble_swinL_uper_split123_pseudo_update.csv'), index=False)